<a href="https://colab.research.google.com/github/zmaruniakova/clever_maps/blob/master/RFM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prerequisites**

## **Inputs**: RFM stands for recency, frequency, monetary value

*   Recency: How recently has the customer made a transaction
*   Frequency: How frequent is the customer in ordering/buying products
*   Monetary: How much does the customer spend on purchasing products

**Data:**
*   monetary value of orders, id (Email), date

# **I. Import libraries**

*  mount google drive content (only for Colab notebook)

*  import libraries(Pandas, Numpy, DateTime and Calendar) and then read the data in the dataframe




In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
class display(object):
    template = """<div style="float: left; padding: 10px;">
    <p style = 'font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    
    def __init__(self, *a):
        self.args = a
    
    def _repr_html_(self):
        return '\n'.join(self.template.format(a,eval(a)._repr_html_())
                        for a in self.args)
    def __repr__(self):
        return '\n\n'.join(a+'\n'+repr(eval(a))
                          for a in self.args)
        

In [112]:
import pandas as pd
import calendar 
from datetime import datetime
import numpy as np

In [113]:
df = pd.read_csv('drive/My Drive/Colab Notebooks/01_RFM_master/data_rfm.csv')
df

,order_id,date,monetary_value
0,99820000000000014798,22.12.2020 11:53:23,264.7
1,99820000000000014798,8.4.2021 17:47:48,306
2,99820000000000014798,11.5.2021 11:38:27,326.38
3,99820000000000014798,2.6.2021 11:32:37,173
4,99820000000000014798,15.6.2021 12:40:51,147
...,...,...,...
323275,99820000001001574601,8.1.2021 13:44:31,496.27
323276,99820000001001574601,9.1.2021 11:19:19,394.73
323277,99820000001001574601,14.1.2021 15:31:21,617.98
323278,99820000001001574601,21.1.2021 11:34:28,682.68


# **II. Data preprocessing**

* use df.info() to chceck your data
* this method prints information about a DataFrame including the index dtype and columns, non-null values and memory usage. 

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323280 entries, 0 to 323279
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   order_id        323280 non-null  object        
 1   date            323280 non-null  object        
 2   monetary_value  323280 non-null  float64       
 3   date_time       323280 non-null  datetime64[ns]
 4   recency         323280 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 12.3+ MB


* keep only date part when using pd.to_datetime (we need day-level purchases for our calculations)


In [115]:
df['date_time'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M:%S')
df['date'] = df['date_time'].dt.date
df

,order_id,date,monetary_value,date_time
0,99820000000000014798,2020-12-22,264.7,2020-12-22 11:53:23
1,99820000000000014798,2021-04-08,306,2021-04-08 17:47:48
2,99820000000000014798,2021-05-11,326.38,2021-05-11 11:38:27
3,99820000000000014798,2021-06-02,173,2021-06-02 11:32:37
4,99820000000000014798,2021-06-15,147,2021-06-15 12:40:51
...,...,...,...,...
323275,99820000001001574601,2021-01-08,496.27,2021-01-08 13:44:31
323276,99820000001001574601,2021-01-09,394.73,2021-01-09 11:19:19
323277,99820000001001574601,2021-01-14,617.98,2021-01-14 15:31:21
323278,99820000001001574601,2021-01-21,682.68,2021-01-21 11:34:28


* find the last purchase and count recency with date_difference function


In [116]:
df['date'].max()

datetime.date(2021, 10, 31)

In [117]:
def date_difference(date):
    date_format = "%Y-%m-%d"

    a = datetime.strptime('2021-10-31',date_format) #last purchase
    b = datetime.strptime(str(date),date_format)
    delta = a-b
    return(delta.days)

In [118]:
df['recency']= df['date'].apply(lambda x:date_difference(x))

df

,order_id,date,monetary_value,date_time,recency
0,99820000000000014798,2020-12-22,264.7,2020-12-22 11:53:23,313
1,99820000000000014798,2021-04-08,306,2021-04-08 17:47:48,206
2,99820000000000014798,2021-05-11,326.38,2021-05-11 11:38:27,173
3,99820000000000014798,2021-06-02,173,2021-06-02 11:32:37,151
4,99820000000000014798,2021-06-15,147,2021-06-15 12:40:51,138
...,...,...,...,...,...
323275,99820000001001574601,2021-01-08,496.27,2021-01-08 13:44:31,296
323276,99820000001001574601,2021-01-09,394.73,2021-01-09 11:19:19,295
323277,99820000001001574601,2021-01-14,617.98,2021-01-14 15:31:21,290
323278,99820000001001574601,2021-01-21,682.68,2021-01-21 11:34:28,283


* retype all columns that are not float by default - .info() func

In [119]:
df['monetary_value'] = df['monetary_value'].str.replace(',', '')
df['monetary_value'] = df['monetary_value'].astype(float)

* group data according to last purchase on customer_id level and also sum monetary value for each row

In [120]:
rfm_df = df.groupby(['order_id']).agg(
                 {
                 'recency':'min',    # recency
                 'monetary_value': 'sum', # monetary
                 'date':'count' #frequency
                 }).reset_index()

rfm_df

,order_id,recency,monetary_value,date
0,99820000000000014798,17,2626.83,11
1,99820000000000015940,67,592.27,1
2,99820000000000021074,192,74.80,1
3,99820000000000022960,325,798.88,1
4,99820000000000030907,215,1355.42,3
...,...,...,...,...
15781,99820000001001571287,135,1226.00,3
15782,99820000001001572352,142,2316.00,2
15783,99820000001001572600,137,496.00,1
15784,99820000001001572953,139,1692.00,3


In [121]:
#find for each column right name in rfm keys
rfm_df = rfm_df.rename({'order_id': 'id', 'recency': 'r', 'date': 'f', 'monetary_value':'m'}, axis=1)
rfm_df['m'] = rfm_df['m'].round(0)
rfm_df

,id,r,m,f
0,99820000000000014798,17,2627.0,11
1,99820000000000015940,67,592.0,1
2,99820000000000021074,192,75.0,1
3,99820000000000022960,325,799.0,1
4,99820000000000030907,215,1355.0,3
...,...,...,...,...
15781,99820000001001571287,135,1226.0,3
15782,99820000001001572352,142,2316.0,2
15783,99820000001001572600,137,496.0,1
15784,99820000001001572953,139,1692.0,3


# **III. RFM analysis**

* define your criteria to fit data distributions 

In [122]:
def f(row):
    if row['f']  in range(0,2):
        val = 1
    elif row['f'] in range(2,10):
        val = 2
    elif row['f'] in range(10,20):
        val = 3
    elif row['f'] in range(20,50):
        val = 4
    else:
        val = 5
    return val

def r(row):
    if row['r']  >= 200:
        val = 1
    elif row['r'] in range(120,200):
        val = 2
    elif row['r'] in range(60,120):
        val = 3
    elif row['r'] in range(35,60):
        val = 4
    else:
        val = 5
    return val

def m(row):
    if row['m'] in np.arange(0,500):
        val = 1
    elif row['m'] in np.arange(500,5000):
        val = 2
    elif row['m'] in np.arange(5000,20000):
        val = 3
    elif row['m'] in np.arange(20000,50000):
        val = 4
    else:
        val = 5
    return val

* use this functions to score your data

In [123]:
rfm_df.loc[:,'F']= rfm_df.apply(f, axis=1)
rfm_df.loc[:,'R']= rfm_df.apply(r, axis=1)
rfm_df.loc[:,'M']= rfm_df.apply(m, axis=1)
rfm_df['RFM_Score'] = rfm_df['R'].map(str) + rfm_df['F'].map(str) + rfm_df['M'].map(str)

rfm_df

,id,r,m,f,F,R,M,RFM_Score
0,99820000000000014798,17,2627.0,11,3,5,2,532
1,99820000000000015940,67,592.0,1,1,3,2,312
2,99820000000000021074,192,75.0,1,1,2,1,211
3,99820000000000022960,325,799.0,1,1,1,2,112
4,99820000000000030907,215,1355.0,3,2,1,2,122
...,...,...,...,...,...,...,...,...
15781,99820000001001571287,135,1226.0,3,2,2,2,222
15782,99820000001001572352,142,2316.0,2,2,2,2,222
15783,99820000001001572600,137,496.0,1,1,2,1,211
15784,99820000001001572953,139,1692.0,3,2,2,2,222


* chceck distributions of each score

In [124]:
F = rfm_df.groupby('F')['id'].count().reset_index()
R = rfm_df.groupby('R')['id'].count().reset_index()
M = rfm_df.groupby('M')['id'].count().reset_index()
display('R','F','M')

,R,id
0,1,3314
1,2,2814
2,3,2132
3,4,1321
4,5,6205
,F,id
0,1,4952
1,2,5083
2,3,1605
3,4,2098


* define segmentation map to name customers

In [127]:
#[r][f][m]

segt_map = {
    '[1][1-2][1-2]': 'Hybernující',
    '[1-2][2][1-2]': 'Hrozící odchodem',
    '[2-3][2-3][3-5]': 'Potřebují pozornost',
    '[1-3][3-5][4-5]': 'Pozor nechceme je ztratit',
    '[4-5][5][4-5]' : 'Šampióni',
    '[4-5][3-5][3-5]': 'Lojální',
    '[3-5][1-3][1-3]': 'Vypadá to dobře',
    '[2-5][2-4][3-5]': 'Potenciální stát se lojálními',
    '[4-5][1-2][1-5]': 'Noví zákazníci',
    '[2-4][1-2][1-5]':'Směrující hybernovat',
    '[1-5][1-5][1-5]':'ostatní'
}

rfm_df['Segment'] = rfm_df['RFM_Score'].replace(segt_map, regex=True)
rfm_df.head()

,id,r,m,f,F,R,M,RFM_Score,Segment
0,99820000000000014798,17,2627.0,11,3,5,2,532,Vypadá to dobře
1,99820000000000015940,67,592.0,1,1,3,2,312,Vypadá to dobře
2,99820000000000021074,192,75.0,1,1,2,1,211,Směrující hybernovat
3,99820000000000022960,325,799.0,1,1,1,2,112,Hybernující
4,99820000000000030907,215,1355.0,3,2,1,2,122,Hybernující


In [128]:
rfm_df.groupby('Segment')['id'].count()

Segment
Hrozící odchodem                 1120
Hybernující                      3112
Lojální                          2580
Potenciální stát se lojálními     209
Potřebují pozornost               318
Pozor nechceme je ztratit         109
Směrující hybernovat             1226
Vypadá to dobře                  5078
ostatní                           531
Šampióni                         1503
Name: id, dtype: int64